In [2]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import wrangle
import warnings
warnings.filterwarnings("ignore")

1. Load the tips dataset.

- Create a column named price_per_person. This should be the total bill divided by the party size.
- Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
- Use select k best to select the top 2 features for predicting tip amount. What are they?
- Use recursive feature elimination to select the top 2 features for tip amount. What are they?
- Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [4]:
# load the 'tips' dataset
df = sns.load_dataset('tips')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [10]:
# split the data into training and test sets
train, validate, test = wrangle.split_dataframe(df)

X_train, y_train = train[['total_bill', 'sex', 'smoker', 'day', 'time', 'size']], train.tip
X_validate, y_validate = validate[['total_bill', 'sex', 'smoker', 'day', 'time', 'size']], validate.tip
X_test, y_test = test[['total_bill', 'sex', 'smoker', 'day', 'time', 'size']], test.tip

# print shapes
